In [ ]:
import faiss, torch
import numpy as np
from transformers import AutoTokenizer, AutoModel

embed_model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
embed_tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Tokenize and create embeddings
def embed_text(text):
    tokens = embed_tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        embeddings = embed_model(**tokens).last_hidden_state.mean(dim=1)
    return embeddings

# Embed documents
embeddings = [embed_text(doc['text']).numpy() for doc in dataset]

# Build FAISS index
dim = embeddings[0].shape[1]  # Embedding dimension
index = faiss.IndexFlatL2(dim)
index.add(np.vstack(embeddings))


In [ ]:
from transformers import pipeline

# Load the zero-shot-classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Define relevant topics
possible_labels = ["golden retrievers", "dogs", "cars", "technology", "food"]

# Check the relevance of the query
def check_relevance(query):
    result = classifier(query, candidate_labels=possible_labels)

    # If the score for "golden retrievers" or "dogs" is low, the question is irrelevant
    if result['labels'][0] not in ["golden retrievers", "dogs"]:
        return "This query doesn't seem to be related to Golden Retrievers."

    return None  # If relevant


In [ ]:
import random
from transformers import pipeline

# Load the Hugging Face model (You can replace GPT-2 with any other local or hosted model)
# Use a conversational model like GPT-2 or GPT-4All
generator = pipeline("text-generation", model="gpt2")

# Predefined responses for conversation flow
greetings = ["Hello! How can I help you today?", "Hi! What would you like to know about Golden Retrievers?", "Hey! Ask me anything about Golden Retrievers!"]
farewells = ["Goodbye! Have a great day!", "Bye! Hope I answered your questions!", "See you later!"]
thanks_responses = ["You're welcome!", "Glad I could help!", "No problem!", "Happy to assist!"]

# FAQ about Golden Retrievers
faq = {
    "what is the average lifespan of a golden retriever": "The average lifespan of a Golden Retriever is around 10 to 12 years.",
    "how long do golden retrievers live": "Golden Retrievers typically live for 10 to 12 years, depending on their health and care.",
    "when do golden retrievers stop growing": "Golden Retrievers usually stop growing around 18 to 24 months of age.",
    "how big do golden retrievers get": "Male Golden Retrievers usually weigh between 65 to 75 pounds, while females weigh between 55 to 65 pounds.",
    "how much exercise do golden retrievers need": "Golden Retrievers need at least one hour of exercise per day. This can include walking, running, or playing fetch.",
    "what activities do golden retrievers enjoy": "Golden Retrievers enjoy swimming, fetching, hiking, running, and playing games like tug-of-war.",
    "are golden retrievers good for hiking": "Yes, Golden Retrievers are excellent hiking companions due to their endurance and love for outdoor activities.",
    "what should i feed my golden retriever": "Golden Retrievers should be fed a balanced diet with high-quality dry dog food. The amount depends on the dog's age, size, and activity level.",
    "how much food does a golden retriever need": "An adult Golden Retriever typically needs between 2 to 3 cups of high-quality dog food daily, divided into two meals.",
    "can golden retrievers eat human food": "Some human foods, like chicken, carrots, and apples, are safe for Golden Retrievers, but avoid toxic foods like chocolate, grapes, and onions.",
    "how often should i groom my golden retriever": "Golden Retrievers should be brushed 3 to 5 times a week to prevent matting and reduce shedding.",
    "do golden retrievers shed a lot": "Yes, Golden Retrievers shed heavily, especially during spring and fall. Regular brushing can help control the shedding.",
    "how often should i bathe my golden retriever": "Golden Retrievers should be bathed every 6 to 8 weeks, or more often if they get dirty or start to smell.",
    "are golden retrievers easy to train": "Yes, Golden Retrievers are intelligent and eager to please, making them relatively easy to train with positive reinforcement.",
    "how can i train my golden retriever": "Use positive reinforcement, such as treats and praise, to train your Golden Retriever. They respond well to consistency and patience.",
    "are golden retrievers good with kids": "Yes, Golden Retrievers are known for their friendly and gentle nature, making them great companions for children.",
    "what are common health issues in golden retrievers": "Common health issues include hip dysplasia, elbow dysplasia, cataracts, and heart problems.",
    "can golden retrievers be service dogs": "Yes, Golden Retrievers are commonly used as service dogs due to their intelligence, calm demeanor, and eagerness to help."
}

# Predefined responses for chit-chat and handling unrelated questions
chit_chat_responses = {
    "how are you": "I'm just a chatbot, but I'm doing great! How about you?",
    "what's your name": "I'm the Golden Retriever chatbot! Ask me anything about Golden Retrievers.",
    "what is your favorite color": "I don't have a favorite color, but Golden Retrievers come in beautiful shades of gold!"
}

# Function to retrieve an answer from the FAQ or chit-chat dictionary
def get_response(user_input):
    user_input = user_input.lower().strip()

    # Check if the input is a greeting or farewell
    if any(greet in user_input for greet in ["hi", "hello", "hey"]):
        return random.choice(greetings)
    if any(bye in user_input for bye in ["bye", "goodbye", "see you"]):
        return random.choice(farewells)
    if "thank" in user_input:
        return random.choice(thanks_responses)

    # Check for chit-chat responses
    for phrase in chit_chat_responses:
        if phrase in user_input:
            return chit_chat_responses[phrase]

    # Check if the input is a question related to Golden Retrievers
    for question, answer in faq.items():
        if question in user_input:
            return answer

    # If none of the above, forward to LLM
    return None

# Fallback to LLM for unrelated questions
def llm_fallback(query):
    response = generator(query, max_length=100, num_return_sequences=1)
    return response[0]['generated_text']

# Chatbot loop
def chatbot():
    print("Welcome to the Golden Retriever chatbot! Ask me anything about Golden Retrievers.")
    print("Type 'exit' to end the conversation.")

    while True:
        user_input = input("\nYou: ").strip().lower()

        if user_input == "exit":
            print(random.choice(farewells))
            break

        # Get chatbot response
        response = get_response(user_input)

        if response:
            print(f"\nBot: {response}")
        else:
            # Forward unrelated questions to the LLM
            llm_response = llm_fallback(user_input)
            print(f"\nBot (LLM): {llm_response}")

# Run the chatbot
chatbot()

In [ ]:
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel
import faiss
import numpy as np

# Golden Retriever facts
data = {
    "text": [
        "Golden Retrievers are friendly, intelligent, and devoted dogs. They are one of the most popular dog breeds in the United States.",
        "Golden Retrievers are great family dogs and are very tolerant with children. They are known for their friendly and tolerant attitudes.",
        "The breed was originally developed in Scotland in the mid-19th century to retrieve game during hunting sessions.",
        "Golden Retrievers require regular exercise and enjoy activities like fetching and swimming.",
        "Golden Retrievers have a dense, water-repellant coat that sheds throughout the year, especially in the spring and fall.",
        "They are often used as guide dogs and in search-and-rescue operations due to their intelligence and trainability.",
        "Golden Retrievers are generally healthy, but they can be prone to certain health issues like hip dysplasia and heart problems."
    ]
}

# Create a dataset
dataset = Dataset.from_dict(data)

# Load a pretrained model for sentence embeddings
embed_model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
embed_tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Function to embed text using the model
def embed_text(text):
    tokens = embed_tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        embeddings = embed_model(**tokens).last_hidden_state.mean(dim=1)
    return embeddings

# Embed the dataset texts
embeddings = [embed_text(doc['text']).numpy() for doc in dataset]

# Build FAISS index
dim = embeddings[0].shape[1]  # Embedding dimension
index = faiss.IndexFlatL2(dim)
index.add(np.vstack(embeddings))

# Function to retrieve similar documents based on a query
def retrieve(query, k=3, threshold=35):
    query_embedding = embed_text(query).numpy()
    distances, indices = index.search(query_embedding, k)

    # Check if the minimum distance is below the threshold (indicating relevance)
    if distances[0][0] > threshold:
        print(distances)
        return "The query doesn't seem related to Golden Retrievers."

    return [dataset[int(i)]['text'] for i in indices[0]]


# Example query
query = "I want a golder retreiver"
retrieved_docs = retrieve(query)
print(retrieved_docs)
